In [6]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [8]:
%autoreload
from pathlib import Path
import sys
root = Path().resolve().absolute().parent.parent
print(root)
sys.path.append(str(root))

from src.pipeline import Pipeline, BuilingIdsEnum
pipe = Pipeline()



/Users/matsalexander/Desktop/SolarEnergyImpact


In [9]:
building_a = pipe.get_data(BuilingIdsEnum.A)
building_b = pipe.get_data(BuilingIdsEnum.B)
building_c = pipe.get_data(BuilingIdsEnum.C)
combined_df = pd.concat([building_a, building_b, building_c])
# reset index
combined_df = combined_df.reset_index(drop=True)
# Perform the train-test split with stratification based on 'building_id'
train_data, test_data = train_test_split(
    combined_df,
    test_size=0.20,
    random_state=42,
    stratify=combined_df['building']
)

# select features
features = ["timestamp", "area", "temperature", "wind_speed", "cloud_fraction", "precipitation"]
target = "value_import"

train_data = train_data[features + [target]]
test_data = test_data[features + [target]]



In [10]:
train_data.head(100)

,timestamp,area,temperature,wind_speed,cloud_fraction,precipitation,value_import
21842,2023-08-16 04:00:00,1384,15.8,0.7,0.9,0.0,44.20
28067,2024-05-01 13:00:00,1384,19.7,7.0,0.0,0.0,28.30
18388,2024-05-30 05:00:00,1095,12.0,2.3,0.9,0.0,17.92
26603,2024-03-01 13:00:00,1384,5.2,3.2,1.0,0.0,47.50
24505,2023-12-05 03:00:00,1384,-7.7,4.0,0.5,0.0,31.90
3265,2023-11-14 01:00:00,1167,-0.6,3.3,0.9,0.0,23.20
5524,2024-02-16 04:00:00,1167,0.1,1.0,1.0,1.1,25.20
9126,2024-07-15 06:00:00,1167,14.2,1.8,1.0,0.1,34.40
14733,2023-12-29 22:00:00,1095,-10.4,0.5,0.9,0.0,41.60
10130,2024-08-26 02:00:00,1167,9.7,3.0,1.0,0.4,21.40


In [11]:
test_data.head(100)

,timestamp,area,temperature,wind_speed,cloud_fraction,precipitation,value_import
9584,2024-08-03 08:00:00,1167,18.800000,0.400000,0.900000,0.0,40.20
11371,2023-08-11 20:00:00,1095,15.700000,1.400000,0.100000,0.0,37.60
27936,2024-04-26 02:00:00,1384,3.600000,3.500000,0.900000,0.0,31.40
17629,2024-04-28 14:00:00,1095,4.000000,2.800000,1.000000,1.4,29.36
26934,2024-03-15 08:00:00,1384,5.000000,1.700000,1.000000,1.8,49.40
22651,2023-09-18 21:00:00,1384,16.500000,6.300000,0.900000,0.0,65.30
6907,2024-04-13 19:00:00,1167,8.800000,1.600000,0.900000,0.0,36.80
18189,2024-05-21 22:00:00,1095,11.900000,0.800000,0.000000,0.0,36.24
17999,2024-05-14 00:00:00,1095,7.800000,0.500000,0.700000,0.0,20.88
11852,2023-08-31 21:00:00,1095,11.600000,0.600000,0.700000,0.0,35.68


In [ ]:
# Train the model using AutoGluon
predictor = TabularPredictor(label=target, eval_metric='mean_absolute_error').fit(
    train_data, 
    presets='best_quality',
    excluded_model_types=['KNN']
    )

No path specified. Models will be saved in: "AutogluonModels/ag-20241109_111950"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       15.00 GB / 32.00 GB (46.9%)
Disk Space Avail:   621.09 GB / 926.35 GB (67.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal 

[1000]	valid_set's l1: 6.07501
[2000]	valid_set's l1: 5.78648
[3000]	valid_set's l1: 5.6268
[4000]	valid_set's l1: 5.52069
[5000]	valid_set's l1: 5.44529
[6000]	valid_set's l1: 5.38433
[7000]	valid_set's l1: 5.34409
[8000]	valid_set's l1: 5.30283
[9000]	valid_set's l1: 5.273
[10000]	valid_set's l1: 5.2553
[1000]	valid_set's l1: 6.08035
[2000]	valid_set's l1: 5.80636
[3000]	valid_set's l1: 5.64797
[4000]	valid_set's l1: 5.53999
[5000]	valid_set's l1: 5.46072
[6000]	valid_set's l1: 5.40086


	Ran out of time, early stopping on iteration 6235. Best iteration is:
	[6235]	valid_set's l1: 5.38999


[1000]	valid_set's l1: 5.94994
[2000]	valid_set's l1: 5.68865
[3000]	valid_set's l1: 5.55142
[4000]	valid_set's l1: 5.45328
[5000]	valid_set's l1: 5.38454
[6000]	valid_set's l1: 5.32817


	Ran out of time, early stopping on iteration 6368. Best iteration is:
	[6364]	valid_set's l1: 5.30857


...

[1000]	valid_set's l1: 6.05065
[2000]	valid_set's l1: 5.7566
[3000]	valid_set's l1: 5.58677
[4000]	valid_set's l1: 5.47299
[5000]	valid_set's l1: 5.39058
[6000]	valid_set's l1: 5.33536
[7000]	valid_set's l1: 5.28602
[8000]	valid_set's l1: 5.24346


	Ran out of time, early stopping on iteration 8182. Best iteration is:
	[8182]	valid_set's l1: 5.23508


[1000]	valid_set's l1: 5.99279
[2000]	valid_set's l1: 5.71728
[3000]	valid_set's l1: 5.54405
[4000]	valid_set's l1: 5.43052
[5000]	valid_set's l1: 5.35591
[6000]	valid_set's l1: 5.28894
[7000]	valid_set's l1: 5.25168
[8000]	valid_set's l1: 5.21839
[9000]	valid_set's l1: 5.18831


	Ran out of time, early stopping on iteration 9791. Best iteration is:
	[9716]	valid_set's l1: 5.16993
	-5.2741	 = Validation score   (-mean_absolute_error)
	571.83s	 = Training   runtime
	2.26s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 24.47s of the 324.58s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 191. Best iteration is:
	[191]	valid_set's l1: 5.07383
	Ran out of time, early stopping on iteration 203. Best iteration is:
	[203]	valid_set's l1: 5.03595
	Ran out of time, early stopping on iteration 239. Best iteration is:
	[239]	valid_set's l1: 5.00877
	Ran out of time, early stopping on iteration 257. Best iteration is:
	[257]	valid_set's l1: 4.9139
	Ran out of time, early stopping on iteration 270. Best iteration is:
	[270]	valid_set's l1: 5.03878
	Ran out of time, early stopping on iteration 285. Best iteration is:
	[285]	valid_set's l

[1000]	valid_set's l1: 5.92245
[2000]	valid_set's l1: 5.62969
[3000]	valid_set's l1: 5.46043
[4000]	valid_set's l1: 5.34185
[5000]	valid_set's l1: 5.25124
[6000]	valid_set's l1: 5.18188
[7000]	valid_set's l1: 5.13049
[8000]	valid_set's l1: 5.0891
[9000]	valid_set's l1: 5.05294
[10000]	valid_set's l1: 5.02345
[1000]	valid_set's l1: 6.0292
[2000]	valid_set's l1: 5.75662
[3000]	valid_set's l1: 5.58846
[4000]	valid_set's l1: 5.47182
[5000]	valid_set's l1: 5.38238
[6000]	valid_set's l1: 5.31468


In [ ]:
# Evaluate on test data
performance = predictor.evaluate(test_data)
# best model: ag-20241022_161331

print("Evaluation Performance:")
performance
# reset index 
test_data = test_data.reset_index(drop=True)
# To see feature importance
global_importance = predictor.feature_importance(test_data)
print("\nFeature Importance:")
print(global_importance)  # Shows which features had the most impact on model predictions


AssertionError: Trainer has no fit models that can infer.

In [ ]:
import numpy as np
matrix = []
size = 1000
for i in range(size):
    matrix.append([])
    for j in range(size):
        K = 1
        if i == j:
            K = 1
        else:
            K = 0
        matrix[i].append(K)
# print as np matrix
print(np.array(matrix))
        

[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


In [ ]:
# model location => AutogluonModels/ag-20241016_095906
main_building = pipe.get_data(BuilingIdsEnum.MAIN)

data_predict = main_building[features]
data_predict


,timestamp,area,temperature,wind_speed,cloud_fraction,precipitation
0,2023-07-01 00:00:00,1199,13.6,1.6,0.5,0.0
1,2023-07-01 01:00:00,1199,13.2,2.0,0.4,0.0
2,2023-07-01 02:00:00,1199,12.3,1.6,0.3,0.0
3,2023-07-01 03:00:00,1199,11.9,0.6,0.9,0.0
4,2023-07-01 04:00:00,1199,11.9,0.2,0.8,0.0
...,...,...,...,...,...,...
10338,2024-09-03 18:00:00,1199,17.8,2.5,1.0,1.4
10339,2024-09-03 19:00:00,1199,17.8,1.8,1.0,2.5
10340,2024-09-03 20:00:00,1199,17.7,1.1,1.0,2.6
10341,2024-09-03 21:00:00,1199,18.0,3.4,1.0,2.9


In [ ]:
prediciton1 = predictor.predict(data_predict)

In [ ]:
# save predicitons as a csv in data folder from root.
import datetime
from pathlib import Path


prediciton1_df = pd.DataFrame(prediciton1)
date_time = datetime.datetime.now().strftime("%Y%m%d")
my_path = Path().resolve().parent.parent / 'data'/ "pred" / date_time / 'prediction_drita.csv'
# create folder
my_path.parent.mkdir(parents=True, exist_ok=True)
prediciton1_df.to_csv(my_path, index=False)